### This project was inspired by the article ["An Analysis of the Top Tech YouTube Channels with Python"](https://towardsdatascience.com/an-analysis-of-the-top-tech-youtube-channels-with-python-ad42c0291723) from Guillaume Weingertner in [Towards Data Science.](https://towardsdatascience.com/)

## Installing the google-api-python-client and Isodate (if necessary)

In [ ]:
#pip install google-api-python-client

In [ ]:
#pip install isodate

## Importing necessary packages
### API configuration

In [17]:
import matplotlib.pyplot as plt
import pandas as pd
from googleapiclient.discovery import build
import isodate

#To get your key you need to go to the Google Developers Console, create a new project and request the 
# key on the "credentials" tab.
api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' 
youtube = build('youtube', 'v3', developerKey=api_key)

## Getting the channel IDs and making a request
### Loop through the IDs to get information from all five channels at once

In [2]:
# Choose whichever channels you like and get their IDs
jauja_id = 'UCVZsY1CF9VwLTKgs9YLQkXw'
vicky_id = 'UCewhO42h_TIKZdntpI-v39g'
saz_sabores_id = 'UCN3Grwgu5ZFNZzkl4IY3V1w'
anna_id = 'UCVf-uzV0XR-3DLtRs1sg6Og'
loli_id = 'UC5ONfXPjWgqElh0NZaRJ1tg'

channel_ids = [jauja_id, vicky_id, saz_sabores_id, anna_id, loli_id]

# Empty list to store channel_overview dictionaries
channel_overviews = []

# Loop through the channel IDs and collect information for each channel
for channel_id in channel_ids:
    request = youtube.channels().list(part="snippet,contentDetails,statistics", id=channel_id)
    response_one = request.execute()

    channel_overview = {
        'title': response_one['items'][0]['snippet']['title'],
        'description': response_one['items'][0]['snippet']['description'],
        'publishedAt': response_one['items'][0]['snippet']['publishedAt'],
        'viewCount': response_one['items'][0]['statistics']['viewCount'],
        'subscriberCount': response_one['items'][0]['statistics']['subscriberCount'],
        'videoCount': response_one['items'][0]['statistics']['videoCount'],
        'uploads': response_one['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    }

    channel_overviews.append(channel_overview)


### Create a DataFrame from the list of channel_overview dictionaries

In [3]:
df_channel_overviews = pd.DataFrame(channel_overviews)
df_channel_overviews

,title,description,publishedAt,viewCount,subscriberCount,videoCount,uploads
0,Jauja Cocina Mexicana,Jauja Cocina Mexicana™ es un canal de cocina ú...,2014-03-19T13:43:10Z,1446521856,8220000,764,UUVZsY1CF9VwLTKgs9YLQkXw
1,VICKY RECETA FACIL,¡¡Nuevas recetas cada semana!! Estás en el mej...,2011-12-16T20:54:25Z,892974996,6100000,1108,UUewhO42h_TIKZdntpI-v39g
2,Temperos e Sabores,"Recetas caseras y muy sabrosas, vamos a cocina...",2012-10-28T17:02:48Z,531060907,4600000,1459,UUN3Grwgu5ZFNZzkl4IY3V1w
3,Anna recetasfaciles,Recetas de cocina fáciles.,2012-11-24T16:50:53Z,651948293,4660000,1101,UUVf-uzV0XR-3DLtRs1sg6Og
4,La Cocina de Loli Dominguez,"Dedicado a gastronomía, cocina y repostería, e...",2013-06-13T17:45:25Z,371275053,2580000,1021,UU5ONfXPjWgqElh0NZaRJ1tg


## Getting information about the videos from each channel

Since the response will only contain max. 50 results, we need to get a nextPageToken and write a while loop to be able to get all videos from the channels. We will loop again through the channel IDs to collect information from each channel.

In [4]:
# Initialize an empty list to store video details
videos = []

# Loop through the channel IDs and collect video information for each channel
for x in range(len(channel_ids)):
    playlistId = df_channel_overviews['uploads'].iloc[x]

    video_ids = []

    request = youtube.playlistItems().list(part="snippet,contentDetails", playlistId=playlistId, maxResults=50)
    response = request.execute()
    nextPageToken = response.get('nextPageToken')

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    while nextPageToken is not None:
        request = youtube.playlistItems().list(part="snippet,contentDetails", playlistId=playlistId, maxResults=50, 
                                               pageToken=nextPageToken)
        response = request.execute()
        nextPageToken = response.get('nextPageToken')
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
    
    # Put video details in data frame
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(part="snippet,contentDetails,statistics", id=video_ids[i:i+50])
        response = request.execute()

        for item in response['items']:
            video = {
                'channelTitle': response['items'][0]['snippet']['channelTitle'],
                'videoId': item['id'],
                'categoryId': item['snippet']['categoryId'],
                'publishedAt': item['snippet']['publishedAt'],
                'title': item['snippet']['title'],
                'description': item['snippet']['description'],
                'tags': item['snippet'].get('tags', 'no_tags'),
                'duration': item['contentDetails']['duration'],
                'viewCount': item['statistics'].get('viewCount', 0),
                'likeCount': item['statistics'].get('likeCount', 0),
                'commentCount': item['statistics'].get('commentCount', 0)
            }
            videos.append(video)

### Creating DataFrame from the list of video details

In [5]:
df_videos = pd.DataFrame(videos)
df_videos

,channelTitle,videoId,categoryId,publishedAt,title,description,tags,duration,viewCount,likeCount,commentCount
0,Jauja Cocina Mexicana,8yp_f0vP3JM,26,2023-09-12T15:27:02Z,¿Qué Hago de Comer el Día del Grito?,Qué Hago de Comer. Que hago de comer con Jauja...,"[que hago de comer, que hago de comer recetas,...",PT14M49S,392079,15845,568
1,Jauja Cocina Mexicana,7hoK_K3KX9Q,26,2023-09-05T16:37:01Z,Mi Nueva Receta de Pozole Rojo. ¡Mejor que Nunca!,Pozole Rojo. Pozole Rojo de Jauja Cocina Mexic...,"[pozole rojo, pozole rojo receta, receta de po...",PT9M39S,646269,29953,1000
2,Jauja Cocina Mexicana,aBU8NmFii3g,26,2023-08-29T15:30:00Z,Entomatadas Rellenas Que No Tienen Comparación,Entomatadas. Entomatadas de Jauja Cocina Mexic...,"[entomatadas, entomatadas receta, receta de en...",PT7M31S,163424,12383,570
3,Jauja Cocina Mexicana,mtyfxC_wSI0,26,2023-08-22T15:28:03Z,Mi Ensalada de Atún Favorita del Camarógrafo,Ensalada de Atún. Ensalada de Atún de Jauja Co...,"[ensalada de atun, ensalada de atun receta, re...",PT6M5S,283617,19575,1006
4,Jauja Cocina Mexicana,wSdzd9xlxNk,26,2023-08-15T15:28:15Z,Miren Las Tortitas de Pollo que Cociné Con Sól...,Tortitas de Pollo. Tortitas de Pollo deshebrad...,"[tortitas de pollo, tortitas de pollo receta, ...",PT8M45S,254356,13133,554
...,...,...,...,...,...,...,...,...,...,...,...
5450,La Cocina de Loli Dominguez,oAjNZKv7rz8,22,2013-09-30T20:29:30Z,"Bizcocho ""Corona de Naranja"" - La cocina de Lo...","Bizcocho ""Corona de Naranja"" una delicia para ...","[Bizcochos, Recetario, reposteria, España, rec...",PT7M2S,30701,656,45
5451,La Cocina de Loli Dominguez,VMPNV5i0utA,22,2013-09-29T14:33:03Z,Rabo de Toro - La cocina de Loli Domínguez - L...,"El rabo de toro es un plato típico Cordobés, s...","[Rabo de toro, rabo, toro, La cocina de Loli D...",PT7M27S,80848,1523,43
5452,La Cocina de Loli Dominguez,Qd0uYn8bmHY,22,2013-09-22T16:42:16Z,Lomo relleno a la miel - La cocina de Loli Dom...,El Lomo relleno a la miel. se puede rellenar ...,"[Lomo relleno a la miel, Lomo relleno, lomo, m...",PT7M34S,43042,769,27
5453,La Cocina de Loli Dominguez,t5oweMSRMok,22,2013-09-15T17:20:33Z,"Receta All i Pebre - Recetas de cocina, paso a...","Es un plato típicamente valenciano , esta es u...","[Recetas, Comida, Province Of Valencia Spanish...",PT5M18S,143758,961,43


### Making some preliminary cleaning before saving to .csv files
#### We start with 'df_channel_overviews'

In [6]:
df_channel_overviews.dtypes

title              object
description        object
publishedAt        object
viewCount          object
subscriberCount    object
videoCount         object
uploads            object
dtype: object

Starting with our df_channel_overviews, we see that all the columns are of the 'object' datatype. We want to convert some of 
them to numeric to be able to make some calculation and visualizations. 

In [7]:
columns_to_numeric = ['viewCount', 'subscriberCount', 'videoCount']
df_channel_overviews[columns_to_numeric] = df_channel_overviews[columns_to_numeric].apply(pd.to_numeric, errors='coerce')

In [8]:
#to avoid scientific notation, we set the display options
pd.options.display.float_format = '{:.2f}'.format

#### Changing the column names to snake_case (personal preference).

In [11]:
new_column_names = {'title': 'channel_name', 'description': 'channel_description', 'publishedAt': 'channel_creation',
                    'viewCount': 'view_count', 'subscriberCount': 'subscriber_count', 'videoCount': 'video_count'}

df_channel_overviews.rename(columns=new_column_names, inplace=True)
df_channel_overviews

,channel_name,channel_description,channel_creation,view_count,subscriber_count,video_count,uploads
0,Jauja Cocina Mexicana,Jauja Cocina Mexicana™ es un canal de cocina ú...,2014-03-19T13:43:10Z,1446521856,8220000,764,UUVZsY1CF9VwLTKgs9YLQkXw
1,VICKY RECETA FACIL,¡¡Nuevas recetas cada semana!! Estás en el mej...,2011-12-16T20:54:25Z,892974996,6100000,1108,UUewhO42h_TIKZdntpI-v39g
2,Temperos e Sabores,"Recetas caseras y muy sabrosas, vamos a cocina...",2012-10-28T17:02:48Z,531060907,4600000,1459,UUN3Grwgu5ZFNZzkl4IY3V1w
3,Anna recetasfaciles,Recetas de cocina fáciles.,2012-11-24T16:50:53Z,651948293,4660000,1101,UUVf-uzV0XR-3DLtRs1sg6Og
4,La Cocina de Loli Dominguez,"Dedicado a gastronomía, cocina y repostería, e...",2013-06-13T17:45:25Z,371275053,2580000,1021,UU5ONfXPjWgqElh0NZaRJ1tg


#### Creating a new column 'mean_views_per_video'.

In [12]:
df_channel_overviews['mean_views_per_video'] = df_channel_overviews['view_count'] / df_channel_overviews['video_count']
df_channel_overviews

,channel_name,channel_description,channel_creation,view_count,subscriber_count,video_count,uploads,mean_views_per_video
0,Jauja Cocina Mexicana,Jauja Cocina Mexicana™ es un canal de cocina ú...,2014-03-19T13:43:10Z,1446521856,8220000,764,UUVZsY1CF9VwLTKgs9YLQkXw,1893353.21
1,VICKY RECETA FACIL,¡¡Nuevas recetas cada semana!! Estás en el mej...,2011-12-16T20:54:25Z,892974996,6100000,1108,UUewhO42h_TIKZdntpI-v39g,805934.11
2,Temperos e Sabores,"Recetas caseras y muy sabrosas, vamos a cocina...",2012-10-28T17:02:48Z,531060907,4600000,1459,UUN3Grwgu5ZFNZzkl4IY3V1w,363989.66
3,Anna recetasfaciles,Recetas de cocina fáciles.,2012-11-24T16:50:53Z,651948293,4660000,1101,UUVf-uzV0XR-3DLtRs1sg6Og,592141.96
4,La Cocina de Loli Dominguez,"Dedicado a gastronomía, cocina y repostería, e...",2013-06-13T17:45:25Z,371275053,2580000,1021,UU5ONfXPjWgqElh0NZaRJ1tg,363638.64


#### Converting 'channel_creation' to datetime and extracting the year. 

In [13]:
df_channel_overviews['channel_creation'] = pd.to_datetime(df_channel_overviews['channel_creation'])

# Extract the year and create a new 'yearPublished' column
df_channel_overviews['year_created'] = df_channel_overviews['channel_creation'].dt.year
df_channel_overviews

,channel_name,channel_description,channel_creation,view_count,subscriber_count,video_count,uploads,mean_views_per_video,year_created
0,Jauja Cocina Mexicana,Jauja Cocina Mexicana™ es un canal de cocina ú...,2014-03-19 13:43:10+00:00,1446521856,8220000,764,UUVZsY1CF9VwLTKgs9YLQkXw,1893353.21,2014
1,VICKY RECETA FACIL,¡¡Nuevas recetas cada semana!! Estás en el mej...,2011-12-16 20:54:25+00:00,892974996,6100000,1108,UUewhO42h_TIKZdntpI-v39g,805934.11,2011
2,Temperos e Sabores,"Recetas caseras y muy sabrosas, vamos a cocina...",2012-10-28 17:02:48+00:00,531060907,4600000,1459,UUN3Grwgu5ZFNZzkl4IY3V1w,363989.66,2012
3,Anna recetasfaciles,Recetas de cocina fáciles.,2012-11-24 16:50:53+00:00,651948293,4660000,1101,UUVf-uzV0XR-3DLtRs1sg6Og,592141.96,2012
4,La Cocina de Loli Dominguez,"Dedicado a gastronomía, cocina y repostería, e...",2013-06-13 17:45:25+00:00,371275053,2580000,1021,UU5ONfXPjWgqElh0NZaRJ1tg,363638.64,2013


### Creating a new column 'view_subscriber_ratio'

In [24]:
df_channel_overviews['view_subscriber_ratio'] = (df_channel_overviews['subscriber_count'] / df_channel_overviews['view_count'])*100
df_channel_overviews

,channel_name,channel_description,channel_creation,view_count,subscriber_count,video_count,uploads,mean_views_per_video,year_created,view_subscriber_ratio
0,Jauja Cocina Mexicana,Jauja Cocina Mexicana™ es un canal de cocina ú...,2014-03-19 13:43:10+00:00,1446521856,8220000,764,UUVZsY1CF9VwLTKgs9YLQkXw,1893353.21,2014,0.57
1,VICKY RECETA FACIL,¡¡Nuevas recetas cada semana!! Estás en el mej...,2011-12-16 20:54:25+00:00,892974996,6100000,1108,UUewhO42h_TIKZdntpI-v39g,805934.11,2011,0.68
2,Temperos e Sabores,"Recetas caseras y muy sabrosas, vamos a cocina...",2012-10-28 17:02:48+00:00,531060907,4600000,1459,UUN3Grwgu5ZFNZzkl4IY3V1w,363989.66,2012,0.87
3,Anna recetasfaciles,Recetas de cocina fáciles.,2012-11-24 16:50:53+00:00,651948293,4660000,1101,UUVf-uzV0XR-3DLtRs1sg6Og,592141.96,2012,0.71
4,La Cocina de Loli Dominguez,"Dedicado a gastronomía, cocina y repostería, e...",2013-06-13 17:45:25+00:00,371275053,2580000,1021,UU5ONfXPjWgqElh0NZaRJ1tg,363638.64,2013,0.69


### Now we are ready to save as a .csv file so we do not have to call the api everytime.

In [25]:
df_channel_overviews.to_csv('channel_overviews.csv', index=False)

### Preliminary cleaning of df_videos

In [15]:
df_videos.dtypes

channelTitle    object
videoId         object
categoryId      object
publishedAt     object
title           object
description     object
tags            object
duration        object
viewCount       object
likeCount       object
commentCount    object
dtype: object

#### Again, we have some object columns we need to convert to numeric and in the case of the duration column, we will create a new column 'duration_sec' with the duration of the videos in seconds. 

In [18]:
# Convert duration column to seconds
df_videos['duration_sec'] = df_videos['duration'].apply(lambda x: isodate.parse_duration(x).total_seconds())

# Convert specific columns to numeric type
numeric_columns = ['viewCount', 'likeCount', 'commentCount', 'duration_sec']
df_videos[numeric_columns] = df_videos[numeric_columns].apply(pd.to_numeric, errors='coerce')

#### Converting 'publishedAt' to datetime and extracting the year into  a new column

In [19]:
# Convert column to Datetime to access the year
df_videos['publishedAt'] = pd.to_datetime(df_videos['publishedAt'])
df_videos['year'] = df_videos['publishedAt'].dt.year
df_videos

,channelTitle,videoId,categoryId,publishedAt,title,description,tags,duration,viewCount,likeCount,commentCount,duration_sec,year
0,Jauja Cocina Mexicana,8yp_f0vP3JM,26,2023-09-12 15:27:02+00:00,¿Qué Hago de Comer el Día del Grito?,Qué Hago de Comer. Que hago de comer con Jauja...,"[que hago de comer, que hago de comer recetas,...",PT14M49S,392079,15845,568,889.00,2023
1,Jauja Cocina Mexicana,7hoK_K3KX9Q,26,2023-09-05 16:37:01+00:00,Mi Nueva Receta de Pozole Rojo. ¡Mejor que Nunca!,Pozole Rojo. Pozole Rojo de Jauja Cocina Mexic...,"[pozole rojo, pozole rojo receta, receta de po...",PT9M39S,646269,29953,1000,579.00,2023
2,Jauja Cocina Mexicana,aBU8NmFii3g,26,2023-08-29 15:30:00+00:00,Entomatadas Rellenas Que No Tienen Comparación,Entomatadas. Entomatadas de Jauja Cocina Mexic...,"[entomatadas, entomatadas receta, receta de en...",PT7M31S,163424,12383,570,451.00,2023
3,Jauja Cocina Mexicana,mtyfxC_wSI0,26,2023-08-22 15:28:03+00:00,Mi Ensalada de Atún Favorita del Camarógrafo,Ensalada de Atún. Ensalada de Atún de Jauja Co...,"[ensalada de atun, ensalada de atun receta, re...",PT6M5S,283617,19575,1006,365.00,2023
4,Jauja Cocina Mexicana,wSdzd9xlxNk,26,2023-08-15 15:28:15+00:00,Miren Las Tortitas de Pollo que Cociné Con Sól...,Tortitas de Pollo. Tortitas de Pollo deshebrad...,"[tortitas de pollo, tortitas de pollo receta, ...",PT8M45S,254356,13133,554,525.00,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5450,La Cocina de Loli Dominguez,oAjNZKv7rz8,22,2013-09-30 20:29:30+00:00,"Bizcocho ""Corona de Naranja"" - La cocina de Lo...","Bizcocho ""Corona de Naranja"" una delicia para ...","[Bizcochos, Recetario, reposteria, España, rec...",PT7M2S,30701,656,45,422.00,2013
5451,La Cocina de Loli Dominguez,VMPNV5i0utA,22,2013-09-29 14:33:03+00:00,Rabo de Toro - La cocina de Loli Domínguez - L...,"El rabo de toro es un plato típico Cordobés, s...","[Rabo de toro, rabo, toro, La cocina de Loli D...",PT7M27S,80848,1523,43,447.00,2013
5452,La Cocina de Loli Dominguez,Qd0uYn8bmHY,22,2013-09-22 16:42:16+00:00,Lomo relleno a la miel - La cocina de Loli Dom...,El Lomo relleno a la miel. se puede rellenar ...,"[Lomo relleno a la miel, Lomo relleno, lomo, m...",PT7M34S,43042,769,27,454.00,2013
5453,La Cocina de Loli Dominguez,t5oweMSRMok,22,2013-09-15 17:20:33+00:00,"Receta All i Pebre - Recetas de cocina, paso a...","Es un plato típicamente valenciano , esta es u...","[Recetas, Comida, Province Of Valencia Spanish...",PT5M18S,143758,961,43,318.00,2013


In [20]:
df_videos.dtypes

channelTitle                 object
videoId                      object
categoryId                   object
publishedAt     datetime64[ns, UTC]
title                        object
description                  object
tags                         object
duration                     object
viewCount                     int64
likeCount                     int64
commentCount                  int64
duration_sec                float64
year                          int64
dtype: object

#### Changing the column names to snake_case (personal preference).
#### Dropping 'Category Id' since it is a column I will not be using.

In [22]:
df_videos_column_names = {'channelTitle': 'channel_name', 'videoId': 'video_id', 'publishedAt': 'published_at',
                        'viewCount': 'view_count', 'likeCount': 'like_count', 'commentCount': 'comment_count'}

df_videos.rename(columns=df_videos_column_names, inplace=True)
df_videos = df_videos.drop('categoryId', axis=1)
df_videos

,channel_name,video_id,published_at,title,description,tags,duration,view_count,like_count,comment_count,duration_sec,year
0,Jauja Cocina Mexicana,8yp_f0vP3JM,2023-09-12 15:27:02+00:00,¿Qué Hago de Comer el Día del Grito?,Qué Hago de Comer. Que hago de comer con Jauja...,"[que hago de comer, que hago de comer recetas,...",PT14M49S,392079,15845,568,889.00,2023
1,Jauja Cocina Mexicana,7hoK_K3KX9Q,2023-09-05 16:37:01+00:00,Mi Nueva Receta de Pozole Rojo. ¡Mejor que Nunca!,Pozole Rojo. Pozole Rojo de Jauja Cocina Mexic...,"[pozole rojo, pozole rojo receta, receta de po...",PT9M39S,646269,29953,1000,579.00,2023
2,Jauja Cocina Mexicana,aBU8NmFii3g,2023-08-29 15:30:00+00:00,Entomatadas Rellenas Que No Tienen Comparación,Entomatadas. Entomatadas de Jauja Cocina Mexic...,"[entomatadas, entomatadas receta, receta de en...",PT7M31S,163424,12383,570,451.00,2023
3,Jauja Cocina Mexicana,mtyfxC_wSI0,2023-08-22 15:28:03+00:00,Mi Ensalada de Atún Favorita del Camarógrafo,Ensalada de Atún. Ensalada de Atún de Jauja Co...,"[ensalada de atun, ensalada de atun receta, re...",PT6M5S,283617,19575,1006,365.00,2023
4,Jauja Cocina Mexicana,wSdzd9xlxNk,2023-08-15 15:28:15+00:00,Miren Las Tortitas de Pollo que Cociné Con Sól...,Tortitas de Pollo. Tortitas de Pollo deshebrad...,"[tortitas de pollo, tortitas de pollo receta, ...",PT8M45S,254356,13133,554,525.00,2023
...,...,...,...,...,...,...,...,...,...,...,...,...
5450,La Cocina de Loli Dominguez,oAjNZKv7rz8,2013-09-30 20:29:30+00:00,"Bizcocho ""Corona de Naranja"" - La cocina de Lo...","Bizcocho ""Corona de Naranja"" una delicia para ...","[Bizcochos, Recetario, reposteria, España, rec...",PT7M2S,30701,656,45,422.00,2013
5451,La Cocina de Loli Dominguez,VMPNV5i0utA,2013-09-29 14:33:03+00:00,Rabo de Toro - La cocina de Loli Domínguez - L...,"El rabo de toro es un plato típico Cordobés, s...","[Rabo de toro, rabo, toro, La cocina de Loli D...",PT7M27S,80848,1523,43,447.00,2013
5452,La Cocina de Loli Dominguez,Qd0uYn8bmHY,2013-09-22 16:42:16+00:00,Lomo relleno a la miel - La cocina de Loli Dom...,El Lomo relleno a la miel. se puede rellenar ...,"[Lomo relleno a la miel, Lomo relleno, lomo, m...",PT7M34S,43042,769,27,454.00,2013
5453,La Cocina de Loli Dominguez,t5oweMSRMok,2013-09-15 17:20:33+00:00,"Receta All i Pebre - Recetas de cocina, paso a...","Es un plato típicamente valenciano , esta es u...","[Recetas, Comida, Province Of Valencia Spanish...",PT5M18S,143758,961,43,318.00,2013


### Now we also save our videos data into a .csv file.

In [23]:
df_videos.to_csv('videos_data.csv', index=False)